In [ ]:
%pip install 'unified_planning[grpc]' # para mac, windows es sin comillas

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

zsh:1: no matches found: unified-planning[pyperplan,tamer,plot]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install up-enhsp

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Código para Fast Downward

from unified_planning.shortcuts import *
import unified_planning as up
from unified_planning.io import PDDLWriter


def crear_problema_lightsout(tamaño: int) -> Problem:
    Celda = UserType('Celda')
    problema = Problem('LightsOut')

    estado = Fluent('estado', BoolType(), c=Celda) # Estado de la celda (encendida o apagada)
    vecino = Fluent('vecino', BoolType(), c1=Celda, c2=Celda) # Relación entre celdas (si son vecinas)

    problema.add_fluent(estado, default_initial_value=False)
    problema.add_fluent(vecino, default_initial_value=False)

    pulsar = InstantaneousAction('pulsar', c=Celda)
    c = pulsar.parameter('c')
    pulsar.add_effect(estado(c), Not(estado(c))) # Al pulsar una celda, se cambia su estado
    problema.add_action(pulsar)

    celdas = [Object(f'[{i},{j}]', Celda) for i in range(tamaño) for j in range(tamaño)]
    problema.add_objects(celdas)

    for i in range(tamaño):
        for j in range(tamaño):
            c_actual = next(obj for obj in celdas if obj.name == f'[{i},{j}]')
            vecinos = [] 
            # Se definen los vecinos (arriba, abajo, izquierda, derecha)
            if i > 0: vecinos.append((i-1, j)) 
            if i < tamaño - 1: vecinos.append((i+1, j))
            if j > 0: vecinos.append((i, j-1))
            if j < tamaño - 1: vecinos.append((i, j+1))
            for vi, vj in vecinos:
                c_vecino = next(obj for obj in celdas if obj.name == f'[{vi},{vj}]')
                problema.set_initial_value(vecino(c_actual, c_vecino), True)

    for c1 in celdas:
        for c2 in celdas:
            cond = And(Equals(pulsar.parameter('c'), c1), vecino(c1, c2)) # Condición de que c2 es vecino de c1
            pulsar.add_effect(estado(c2), Not(estado(c2)), cond) # Al pulsar c1, se cambia el estado de c2 si es vecino

    for c_obj in celdas:
        problema.set_initial_value(estado(c_obj), False) # Todas las celdas comienzan apagadas
        problema.add_goal(estado(c_obj)) # El objetivo es que todas estén encendidas

    return problema

def resolver_lightsout(tamaño: int):
    problema = crear_problema_lightsout(tamaño)
    print(f"\nResolviendo tablero de tamaño {tamaño}x{tamaño}...\n")

    # Aquí se generan los archivos PDDL
    w = PDDLWriter(problema, rewrite_bool_assignments=True)
    w.write_domain('Dominio.pddl')
    w.write_problem('Problema.pddl')

# Aquí definimos el tamaño del tablero
resolver_lightsout(tamaño=5)



Resolviendo tablero de tamaño 5x5...



In [21]:
# Búsqueda en anchura con parseo de archivos PDDL

from unified_planning.io import PDDLReader     
from unified_planning.shortcuts import *        
from collections import deque                   

# Extrae la relación de vecindad desde el archivo PDDL del problema
def get_vecinos_from_problem(problem):
    vecinos = {}
    celdas = [str(o) for o in problem.objects(typename=problem.user_types[0])]
    
    for c in celdas:
        vecinos[c] = []

    # Recorremos los valores iniciales buscando los vecinos
    for fluent, val in problem.initial_values.items():
        if 'vecino' in str(fluent) and val.constant_value():
            c1 = str(fluent.args[0])
            c2 = str(fluent.args[1])
            vecinos[str(c1)].append(str(c2))  # Añadimos c2 como vecino de c1
    return vecinos

# Obtiene el estado inicial del tablero a partir del archivo PDDL
def estado_inicial_from_problem(problem):
    estado = {}
    for fluent, val in problem.initial_values.items():
        if 'estado' in str(fluent):
            celda = str(fluent.args[0])
            estado[celda] = val.constant_value()  # True si está encendida, False si está apagada
    return estado

# Aplica la acción de pulsar una celda
def aplicar_accion_estado(estado, celda, vecinos):
    nuevo_estado = estado.copy()
    nuevo_estado[celda] = not nuevo_estado[celda]  # Cambiamos el estado de la celda pulsada
    for v in vecinos[celda]:
        nuevo_estado[v] = not nuevo_estado[v]      # Cambiamos también el de sus vecinos
    return nuevo_estado

# Comprueba si el estado actual cumple el objetivo: todas las luces encendidas
def es_objetivo(estado):
    return all(estado.values())

# Heurística: cuenta cuántas luces están encendidas
def similitud(estado):
    return sum(estado.values())


def busqueda_heuristica_pddl(problem, n=3):
    vecinos = get_vecinos_from_problem(problem)           # Diccionario de vecinos
    estado_inicial = estado_inicial_from_problem(problem) # Diccionario de estado inicial
    celdas = list(vecinos.keys())                          # Lista de celdas a explorar
    cola = deque()
    cola.append((estado_inicial, []))                      # Cola para BFS: (estado actual, plan hasta ahora)
    visitados = set()

    print(vecinos)         
    print(estado_inicial)  

    
    while cola:
        estado_actual, plan = cola.popleft()
        estado_hash = frozenset(estado_actual.items())     # Usamos hash del estado para evitar repeticiones
        if estado_hash in visitados:
            continue
        visitados.add(estado_hash)

        if es_objetivo(estado_actual):                     # Si el estado es el objetivo, retornamos el plan
            return plan

        sucesores = []
        for c in celdas:
            nuevo_estado = aplicar_accion_estado(estado_actual, c, vecinos)
            nuevo_plan = plan + [f'pulsar {c}']
            sucesores.append((nuevo_estado, nuevo_plan))

        # Verificamos si alguno de los nuevos estados ya es solución
        for est, p in sucesores:
            if es_objetivo(est):
                return p

        # Ordenamos sucesores por heurística (mayor número de luces encendidas primero)
        sucesores.sort(key=lambda x: similitud(x[0]), reverse=True)
        cola.extend(sucesores[:n])  # Añadimos solo los n mejores sucesores a la cola

    return None 

# Leemos el problema desde los archivos PDDL generados previamente
reader = PDDLReader()
problema = reader.parse_problem('Dominio.pddl', 'Problema.pddl')

plan = busqueda_heuristica_pddl(problema, n=5)
if plan:
    print(f"Plan encontrado en {len(plan)} pasos:")
    for i, paso in enumerate(plan, 1):
        print(f"Paso {i}: {paso}")
else:
    print("No se encontró plan con esta búsqueda heurística.")

{'o__0_2_': ['o__1_2_', 'o__0_1_', 'o__0_3_'], 'o__0_0_': ['o__1_0_', 'o__0_1_'], 'o__0_4_': ['o__1_4_', 'o__0_3_'], 'o__3_1_': ['o__2_1_', 'o__4_1_', 'o__3_0_', 'o__3_2_'], 'o__1_0_': ['o__0_0_', 'o__2_0_', 'o__1_1_'], 'o__0_1_': ['o__1_1_', 'o__0_0_', 'o__0_2_'], 'o__1_4_': ['o__0_4_', 'o__2_4_', 'o__1_3_'], 'o__3_4_': ['o__2_4_', 'o__4_4_', 'o__3_3_'], 'o__0_3_': ['o__1_3_', 'o__0_2_', 'o__0_4_'], 'o__4_3_': ['o__3_3_', 'o__4_2_', 'o__4_4_'], 'o__4_2_': ['o__3_2_', 'o__4_1_', 'o__4_3_'], 'o__1_1_': ['o__0_1_', 'o__2_1_', 'o__1_0_', 'o__1_2_'], 'o__4_1_': ['o__3_1_', 'o__4_0_', 'o__4_2_'], 'o__1_2_': ['o__0_2_', 'o__2_2_', 'o__1_1_', 'o__1_3_'], 'o__2_2_': ['o__1_2_', 'o__3_2_', 'o__2_1_', 'o__2_3_'], 'o__4_0_': ['o__3_0_', 'o__4_1_'], 'o__4_4_': ['o__3_4_', 'o__4_3_'], 'o__2_0_': ['o__1_0_', 'o__3_0_', 'o__2_1_'], 'o__2_1_': ['o__1_1_', 'o__3_1_', 'o__2_0_', 'o__2_2_'], 'o__1_3_': ['o__0_3_', 'o__2_3_', 'o__1_2_', 'o__1_4_'], 'o__2_3_': ['o__1_3_', 'o__3_3_', 'o__2_2_', 'o__2_4_'], 

In [ ]:
## Búsqueda en anchura desde cero

from collections import deque

# Definimos los vecinos
def crear_vecinos(tamaño): 
    vecinos = {}
    for i in range(tamaño):
        for j in range(tamaño):
            pos = f'[{i},{j}]'
            vecinos[pos] = []
            if i > 0:
                vecinos[pos].append(f'[{i-1},{j}]')
            if i < tamaño - 1:
                vecinos[pos].append(f'[{i+1},{j}]')
            if j > 0:
                vecinos[pos].append(f'[{i},{j-1}]')
            if j < tamaño - 1:
                vecinos[pos].append(f'[{i},{j+1}]')
    return vecinos

def aplicar_accion(estado_actual, celda_pulsada, vecinos):
    nuevo_estado = estado_actual.copy()
    # Esto invierte la celda pulsada
    nuevo_estado[celda_pulsada] = not nuevo_estado[celda_pulsada]
    # Esto invierte los vecinos
    for v in vecinos[celda_pulsada]:
        nuevo_estado[v] = not nuevo_estado[v]
    return nuevo_estado

# Comprueba si el estado actual es el objetivo: todas las celdas encendidas
def es_objetivo(estado_actual):
    return all(estado_actual.values())

# Heurística: cuenta cuántas luces están encendidas
def similitud(estado_actual):
    return sum(estado_actual.values())

# Búsqueda en anchura con heurística
def busqueda_anchura(tamaño, n=3):
    celdas = [f'[{i},{j}]' for i in range(tamaño) for j in range(tamaño)]  
    vecinos = crear_vecinos(tamaño)                                        
    estado_inicial = {c: False for c in celdas}                            

    cola = deque()                                       # Creamos la cola de búsqueda
    cola.append((estado_inicial, []))                    # Añadimos el estado inicial con un plan vacío
    visitados = set()                                    # Conjunto de estados ya explorados

    while cola:
        estado_actual, plan = cola.popleft()             # Sacamos el siguiente estado de la cola
        estado_frozenset = frozenset(estado_actual.items())  # Convertimos a conjunto inmutable para comparar
        if estado_frozenset in visitados:
            continue
        visitados.add(estado_frozenset)                  # Marcamos este estado como visitado

        if es_objetivo(estado_actual):                   # Si es el objetivo, devolvemos el plan
            return plan

        sucesores = []
        for c in celdas:
            nuevo_estado = aplicar_accion(estado_actual, c, vecinos)  # Aplicamos la acción
            nuevo_plan = plan + [f'pulsar {c}']                        # Añadimos la acción al plan
            sucesores.append((nuevo_estado, nuevo_plan))              # Guardamos el nuevo estado y plan

        # Si encontramos el objetivo entre los sucesores, lo devolvemos
        for est, p in sucesores:
            if es_objetivo(est):
                return p

        # Ordenamos los sucesores por su similitud con el objetivo (más luces encendidas)
        sucesores.sort(key=lambda x: similitud(x[0]), reverse=True)
        cola.extend(sucesores[:n])  # Añadimos solo los n mejores sucesores

    return None

plan = busqueda_anchura(3, n=5)
if plan:
    print(f"Plan encontrado en {len(plan)} pasos:")
    for i, paso in enumerate(plan, 1):
        print(f"Paso {i}: {paso}")
else:
    print("No se encontró plan con esta búsqueda heurística.")